In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import mnist
(images, labels), (testX, testY) = mnist.load_data()

images = np.array(images).astype("float").reshape(-1,784) / 255
labels = np.array(labels)
X = images
Y = labels
testX = np.array(testX).astype("float").reshape(-1, 784) / 255
testY= np.array(testY)

testX.shape, testY.shape, X.shape, Y.shape

In [ ]:
# MNIST Digits(or at least pointers to URLs that allow the MNIST data to be loaded)
# is built into tensorflow.  
# Q: HOW DO I INSTALL TENSORFLOW? 
# A: Create a new python environment so the install doesn't break 
# other python package installs that you may care about more.

# https://docs.anaconda.com/anaconda/user-guide/tasks/tensorflow/

conda create -n tf tensorflow # Create new environment
conda activate tf             # Temporarily switch python to use new environment



In [ ]:
# You can find dozens of websites, blogs, and github repositories that 
# do something or other with this dataset.

# One of the first things that they do is change the data type and 
# manipulate the range: 
images = np.array(images).astype("float") / 255
labels = np.array(labels)
X=images
Y=labels
testX = np.array(testX).astype("float") / 255
testY= np.array(testY)

In [ ]:
digits = labels

In [ ]:
digits[0:20]

In [ ]:
digits = np.array(images)

In [ ]:
digits.shape

In [ ]:
p = np.reshape(digits, newshape=( 60000, 28,28))

In [ ]:
p.shape

In [ ]:
plt.imshow(p[4,::])

In [ ]:
plt.subplot(221)
plt.imshow(p[0,::])
plt.subplot(222)
plt.imshow(p[1,::])
plt.subplot(223)
plt.imshow(p[2,::])
plt.subplot(224)
plt.imshow(p[3,::])
plt.savefig("MNIST.png", dpi=300, bbox_inches="tight")

In [ ]:
X.shape, Y.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=500)
lr.fit(X, Y)
lr.score(testX, testY)

In [ ]:
lr

In [ ]:
lr.coef_.shape

In [ ]:
lr.predict_log_proba(testX.reshape((-1,784)))

In [ ]:
def optim0(image):
    #print(type(image))
    #print(image.shape)
    g=lr.predict_log_proba(X[0,:]+image[np.newaxis,:])
    return(g[0][0])

In [ ]:
from scipy.optimize import minimize

init = np.random.random((28,28))
init.shape

In [ ]:
init_flat = init.reshape((1,-1))[0]
init_flat.shape

In [ ]:
type(init_flat), init_flat.shape

In [ ]:
optim0(init_flat)

In [ ]:
t = minimize(optim0, init_flat)

t

In [ ]:
init0 = np.zeros((28,28)).reshape(-1,1)[0]
optim0(init0)

In [ ]:
t = minimize(optim0, init0)


In [ ]:
t

In [ ]:
plt.imshow(t.x.reshape((28,28)))

In [ ]:
X[0,:]
lr.predict(X[0,:].reshape(1,-1))

In [ ]:
lr.predict((t.x+X[0,:]).reshape(1,-1))

In [ ]:
crange = np.exp(np.arange( - 9, 9, 2))
fits = []
for c in crange:
    print(c)
    fits.append(LogisticRegression(C=c, max_iter=100, penalty="l2", solver="liblinear").fit(X, Y))
accuracy = [f.score(testX, testY) for f in fits]

In [ ]:
# Let us compare a highly regularized set of linear regression 
# coefficients with a almost-unregularized set.

plt.subplot(1,2,1)
plt.imshow(fits[0].coef_[9,:].reshape((28,28)))
plt.subplot(1,2,2)
plt.imshow(fits[-1].coef_[9,:].reshape((28,28)))


In [ ]:
# Maybe a diverging colormap would help us here?

plt.subplot(1,2,1)
plt.imshow(fits[1].coef_[0,:].reshape((28,28)), cmap="RdYlGn")
plt.subplot(1,2,2)
plt.imshow(fits[-1].coef_[0,:].reshape((28,28)), cmap="RdYlGn")

In [ ]:
plt.plot(crange, accuracy)
plt.semilogx()

In [ ]:
#  https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

from sklearn.model_selection import cross_val_score
# cross_val_score(<model>, x, y, cv=<fold>) 


In [ ]:
accuracy_cv = [cross_val_score(f, testX, testY) for f in fits]

In [ ]:
accuracy_cv

In [ ]:
accuracy.shape


In [ ]:
np.array(accuracy_cv).shape

In [ ]:
plt.plot(crange, np.array(accuracy_cv).mean(axis=1), label="Cross-validated")
plt.plot(crange, accuracy, label="Training")

plt.xlabel("Regularization parameter")
plt.ylabel("Digit accuracy")
plt.semilogx()

In [ ]:
# What about L1 regularization?  This will effectively throw
# out pixels that have too little influence on the result.
crange2 = np.power(10.0, np.arange( - 5, 5, 1))
fits_l1 = []
for c in crange2:
    print(c)
    fits_l1.append(LogisticRegression(C=c, max_iter=100, penalty="l1", solver="liblinear").fit(X, Y))
accuracy_l1 = [f.score(testX, testY) for f in fits_l1]


In [ ]:
# The regularized parameters should be closer to 0...
plt.hist(fits_l1[-1].coef_[1,:], bins=30)
plt.hist(fits_l1[0].coef_[1,:], bins=30)

In [ ]:
plt.plot(crange2, accuracy_l1, label="l1")
plt.plot(crange, accuracy, label="l2")
plt.semilogx()
plt.ylim((0.8, 0.95))
plt.legend()
plt.xlabel("Regularization parameter c")
plt.ylabel("Accuracy on holdout set")

In [ ]:
fits=fits_l1
for i in range(len(fits)):
    plt.subplot(1, len(fits), i+1)
    plt.imshow(fits_l1[i].coef_[1,:].reshape((28,28)), cmap="RdYlGn")
   # plt.title("C = {:d}".format(crange2))
    plt.axis("off")

In [ ]:
plt.subplot(1,3,1)
plt.imshow(fits_l1[5].coef_[8,:].reshape((28,28)), cmap="RdYlGn")
plt.subplot(1,3,2)
plt.imshow(fits_l1[7].coef_[8,:].reshape((28,28)), cmap="RdYlGn")
plt.subplot(1,3,3)
l=0.03
plt.imshow(np.maximum(-l, np.minimum(l, fits_l1[6].coef_[8,:].reshape((28,28)))), cmap="RdYlGn")

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
svd = TruncatedSVD( 50).fit(X)

In [ ]:
svd.explained_variance_ratio_

In [ ]:
plt.plot(svd.explained_variance_ratio_, 'o')

In [ ]:
svd.components_.shape

In [ ]:
p, a =plt.subplots(1, len(fits))
for i in range(25):
    plt.subplot(5,5 , i+1)
    plt.imshow(svd.components_[i].reshape((28,28)))
    plt.axis("off")

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
svdreg = TruncatedSVD( 50).fit(scaler.transform(X))

In [ ]:
plt.plot(svdreg.explained_variance_ratio_, 'o')

In [ ]:
for i in range(25):
    plt.subplot(5,5 , i+1)
    plt.imshow(svdreg.components_[i].reshape((28,28)))
    plt.axis("off")

In [ ]:
# Keep in mind, these are from the variance of the data--no labels
# were used in constructing these vectors.
# And, this procedure is linear in the data; no regularization 
# was applied, just averaging.

In [ ]:
# Now we can project X onto the PCA components; we do 
# this by matrix multiplication

In [ ]:
svd.components_.shape

In [ ]:
PC = np.dot( X, svd.components_.T)

In [ ]:
PC.shape

In [ ]:
h = {0: "black", 1:"brown", 2:"red", 3:"orange", 4:"yellow",
    5:"green", 6:"blue", 7:"purple", 8:"grey", 9:"white"}
colorlabels = []
for i,l in enumerate(Y):
    colorlabels.append(h[l])


In [ ]:
plt.scatter(PC[0:1000,0], PC[0:1000,1] , c= colorlabels[0:1000])

In [ ]:
# Calculate average digits
avgdigit =[]
stddigit =[]
for i in range(10):
    avgdigit.append(images[np.where(labels==i)].mean(axis=0).reshape((28,28)))
    stddigit.append(images[np.where(labels==i)].std(axis=0).reshape((28,28)))
avgdigit=np.array(avgdigit)
avgdigit.shape

In [ ]:
avgdigit_reshape = avgdigit.reshape((10, -1))

In [ ]:
svd_avg = TruncatedSVD(9).fit(scaler.transform(avgdigit_reshape))

In [ ]:
svd_avg.components_.shape

In [ ]:
PC = np.dot(svd_avg.components_, X.T).T

In [ ]:
plt.scatter(PC[0:1000,0], PC[0:1000,1] , c= colorlabels[0:1000])

In [ ]:
plt.scatter(PC[0:1000,2], PC[0:1000,3] , c= colorlabels[0:1000])

In [ ]:
plt.scatter(PC[0:1000,4], PC[0:1000,5] , c= colorlabels[0:1000])